In [40]:
import torch
import cv2
import os

In [41]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom',path=r'D:\Github\OTUS22\UAVweights300\best.onnx')  # or yolov5n - yolov5x6, custom

Using cache found in C:\Users\enes_/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-12-14 Python-3.10.4 torch-1.12.1 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)

Loading D:\Github\OTUS22\UAVweights300\best.onnx for ONNX Runtime inference...
Adding AutoShape... 


In [42]:
# Inference
model.conf = 0.6 # NMS confidence threshold
#model.iou = 0.45  # NMS IoU threshold
#model.agnostic = False  # NMS class-agnostic
#model.multi_label = False  # NMS multiple labels per box
#model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
model.max_det = 1  # maximum number of detections per image
#model.amp = False  # Automatic Mixed Precision (AMP) inference

In [43]:
# Video address
vid_name = 'F:\SabitKanatVideolar\Labellanacakvideolar\GH010012_Trim.mp4'

# Video Capturer
cap = cv2.VideoCapture(vid_name)

In [44]:
def drawBox(img,bbox):
    x,y,w,h = tuple(map(int,bbox))
    cv2.rectangle(img,(x,y),((x+w),(y+h)),(255,0,255),3,1)
    cv2.putText(img, "Tracking", (75, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

In [45]:
# Options for cv2.rectangle
thickness=1
color=(0,0,255)

# Options for cv2.putText
# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# font
fontScale = 0.5
  

In [46]:
def createTracker(num):
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[num]

    if tracker_type == 'BOOSTING':
        tracker = cv2.legacy.TrackerBoosting_create()
    if tracker_type == 'MIL':
        tracker = cv2.TrackerMIL_create() 
    if tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create() 
    if tracker_type == 'TLD':
        tracker = cv2.legacy.TrackerTLD_create() 
    if tracker_type == 'MEDIANFLOW':
        tracker = cv2.legacy.TrackerMedianFlow_create() 
    if tracker_type == 'GOTURN':
        tracker = cv2.TrackerGOTURN_create()
    if tracker_type == 'MOSSE':
        tracker = cv2.legacy.TrackerMOSSE_create()
    if tracker_type == "CSRT":
        tracker = cv2.TrackerCSRT_create()
    return tracker

In [47]:
cnt = 0
flag=1
while(True):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640,640), interpolation = cv2.INTER_AREA)
    if flag:
        tracker = createTracker(7)
        results = model(frame)
        df = results.pandas().xyxy[0]
        bbox= [20,10,20,10]
        ret = tracker.init(frame, bbox)
        flag =0
    if ret:
        cnt+=1
        if cnt == 5:
            cnt = 0
            results = model(frame)
            df = results.pandas().xyxy[0]
            if df.size != 0:
                frame = cv2.rectangle(frame, (int(df["xmin"]),int(df["ymin"])), (int(df["xmax"]),int(df["ymax"])), color, thickness)
                frame = cv2.putText(frame, 'UAV: {}'.format(df["confidence"]), (int(df["xmin"]),int(df["ymin"])), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
                bbox= (int(df["xmin"]), int(df["ymin"]), int(df["xmax"])- int(df["xmin"]), int(df["ymax"])- int(df["ymin"]))
                tracker = createTracker(2)
                _ = tracker.init(frame, bbox)
        _, bbox = tracker.update(frame)
        
        try:
            drawBox(frame,bbox)
        except:
            frame = cv2.putText(frame, 'No detection', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)

        cv2.imshow("sex", frame)
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    key = cv2.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [48]:
df

,xmin,ymin,xmax,ymax,confidence,class,name
0,297.84964,160.739685,352.701935,193.952576,0.967069,0,UAV
